# QENS data reduction - DMSC summer school

In [ ]:
import os

import numpy as np
import plopp as pp
import scipp as sc
import scippneutron as scn

import qens_utils

In [ ]:
%matplotlib widget

In [ ]:
top_folder = '../data/McStas_QENS_NeXus_updated_counts'
run_folder = 'QENS_elastic_NeXus_1_pulse'
fname = os.path.join(top_folder, run_folder, 'mccode.h5')

In [ ]:
events = qens_utils.load_qens(fname)

In [ ]:
events

In [ ]:
source_settings = {
    'center': events.coords['source_position'],
    'color': '#FFC133',
    'size': sc.vector(value=[10, 20, 10], unit=sc.units.m),
    'type': 'cylinder',
}
sample_settings = {
    'center': events.meta['sample_position'],
    'color': '#000000',
    'wireframe': True,
    'size': sc.vector(value=[0.01, 0.01, 0.01], unit=sc.units.m),
    'type': 'box',
}
analyzer_settings = {
    'center': events.coords['analyzer_position'],
    'color': '#AA0000',
    'size': sc.vector([0.1, 0.1, 0.1], unit='m'),
    'type': 'box',
}
scn.instrument_view(
    events,
    components={'analyzer': analyzer_settings, 'source': source_settings, 'sample': sample_settings})

In [ ]:
def backscattered_l2(position, sample_position, analyzer_position):
    """
    Compute the length of the secondary flight path for backscattering off an analyzer.
    """
    return sc.norm(position-analyzer_position) + sc.norm(analyzer_position-sample_position)


def wavelength_from_analyzer(analyzer_dspacing, analyzer_angle):
    """
    Compute the neutron wavelength after scattering from the analyzer's d-spacing.

    Assuming Bragg scattering in the analyzer, the wavelength is
        wavelength = 2 * d * sin(theta)

    Where
        d is the analyzer's d-spacing,
        theta is the scattering angle or equivalently, the tilt of the analyzer
              w.r.t. to the sample-analyzer axis.
    """
    # 2*theta is the angle between transmitted and scattered beam.
    # So because of backscattering, we need to subtract the analyzer angle from 2*pi.
    return 2 * analyzer_dspacing * sc.sin(sc.scalar(np.pi/2, unit='rad') - analyzer_angle.to(unit='rad'))


def final_energy(final_wavelength):
    """
    Compute the neutron energy after scattering.

    Uses
        final_energy = mn / 2 * final_speed**2
        final_speed = 2 * pi * hbar / mn / final_wavelength

    Where
        mn is the neutron mass,
        final_wavelength is the wavelength after scattering,
        final_speed is the speed after scattering.
    """
    return 4 / sc.constants.neutron_mass * (sc.constants.h / final_wavelength) ** 2

In [ ]:
from scippneutron.conversion.graph.beamline import beamline
from scippneutron.conversion.graph.tof import indirect_inelastic

graph = {
    **beamline(scatter=True),
    **indirect_inelastic('tof'),
    'L2': backscattered_l2,
    'final_wavelength': wavelength_from_analyzer,
    'final_energy': final_energy,
}
del graph['two_theta']
del graph['scattered_beam']
del graph['Ltotal']
sc.show_graph(graph, simplified=True)

In [ ]:
da = events.transform_coords('energy_transfer', graph=graph)